In [115]:
from pyvis.network import Network
import networkx as nx

import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import numpy
import re

In [116]:
url = "http://moss.stanford.edu/results/2/8635424362468"
res = requests.get(url)
dom = BeautifulSoup(res.text, 'lxml')

In [117]:
node_name_p = r'.*/(.*)_.*'
node_name_full_p = r'.*/(.*)\..*'

In [118]:
G = nx.Graph()

In [119]:
rows = dom.select('table tr')  #table
# print(rows)
rows = [row.select('td') for row in rows if row.select('td')]
# print(rows)
row_data = []

In [120]:
max_val = min_val = int(rows[0][2].text.strip())

for data in rows: 
    if (int(data[2].text.strip()) > max_val):
        max_val = int(data[2].text.strip())
    elif (int(data[2].text.strip()) < min_val):
        min_val = int(data[2].text.strip())

# print(min_val)
# print(max_val)

In [121]:
node_size_p  = r"\d+(?=%)"
node_similarity = r"\(([^)]+)\)"

In [125]:
edge_scale = 10
node_size = 20

In [126]:
for data in rows:
    # print(data)
    node1 = re.search(node_name_full_p, data[0].text.strip())[1]
    # print(node1)
    node2 = re.search(node_name_full_p, data[1].text.strip())[1]
    # print(node2)
    node1_similarity = re.search(node_similarity, data[0].text.strip())[1]
    node2_similarity = re.search(node_similarity, data[1].text.strip())[1]

    G.add_node(node1, title=node1, size=node_size)
    G.add_node(node2, title=node2, size=node_size)
    weight = numpy.interp(data[2].text.strip(),[min_val,max_val],[0,1])
    scaled_weight = weight * edge_scale
    title_weight = str(round(weight, 3))
    G.add_edge(node1, node2, weight=scaled_weight, title=f'{node1}: {node1_similarity}\n{node2}: {node2_similarity}\nLines Matched: {data[2].text.strip()}\nWeight: {title_weight}')
    # print(data[2].text.strip())
    # row_data.append([data[0].text.strip(), data[1].text.strip()])


In [127]:
# nt = Network('800px', '800px', select_menu=True, neighborhood_highlight=True)
nt = Network('800px', '800px', neighborhood_highlight=True)
# nt.toggle_physics(True)
nt.repulsion(node_distance=200, spring_length=200)
nt.from_nx(G)
# nt.show_buttons(filter_=True)
nt.show('nx.html')